In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Test SQL app").getOrCreate()



In [2]:
df=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                     user="root",\
                                     password="password",\
                                     url="jdbc:mysql://localhost:3306/classicmodels",\
                                     dbtable="classicmodels.orders").load()



In [3]:
df.count()  # return number of rows



326

In [4]:
df.printSchema()  # return schema

root
 |-- orderNumber: integer (nullable = true)
 |-- orderDate: date (nullable = true)
 |-- requiredDate: date (nullable = true)
 |-- shippedDate: date (nullable = true)
 |-- status: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- customerNumber: integer (nullable = true)



In [5]:
df.show()

+-----------+----------+------------+-----------+-------+--------------------+--------------+
|orderNumber| orderDate|requiredDate|shippedDate| status|            comments|customerNumber|
+-----------+----------+------------+-----------+-------+--------------------+--------------+
|      10100|2003-01-06|  2003-01-13| 2003-01-10|Shipped|                null|           363|
|      10101|2003-01-09|  2003-01-18| 2003-01-11|Shipped|Check on availabi...|           128|
|      10102|2003-01-10|  2003-01-18| 2003-01-14|Shipped|                null|           181|
|      10103|2003-01-29|  2003-02-07| 2003-02-02|Shipped|                null|           121|
|      10104|2003-01-31|  2003-02-09| 2003-02-01|Shipped|                null|           141|
|      10105|2003-02-11|  2003-02-21| 2003-02-12|Shipped|                null|           145|
|      10106|2003-02-17|  2003-02-24| 2003-02-21|Shipped|                null|           278|
|      10107|2003-02-24|  2003-03-03| 2003-02-26|Shipped|Dif

In [7]:
query="(select * from orders where customerNumber = 144) as cust"

df=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                     user="root",\
                                     password="password",\
                                     url="jdbc:mysql://localhost:3306/classicmodels",\
                                     dbtable=query).load()
df.show()
df.explain(True)



+-----------+----------+------------+-----------+-------+--------------------+--------------+
|orderNumber| orderDate|requiredDate|shippedDate| status|            comments|customerNumber|
+-----------+----------+------------+-----------+-------+--------------------+--------------+
|      10112|2003-03-24|  2003-04-03| 2003-03-29|Shipped|Customer requeste...|           144|
|      10320|2004-11-03|  2004-11-13| 2004-11-07|Shipped|                null|           144|
|      10326|2004-11-09|  2004-11-16| 2004-11-10|Shipped|                null|           144|
|      10334|2004-11-19|  2004-11-28|       null|On Hold|The outstaniding ...|           144|
+-----------+----------+------------+-----------+-------+--------------------+--------------+

== Parsed Logical Plan ==
Relation [orderNumber#112,orderDate#113,requiredDate#114,shippedDate#115,status#116,comments#117,customerNumber#118] JDBCRelation((select * from orders where customerNumber = 144) as cust) [numPartitions=1]

== Analyzed L

In [8]:
query="(select * from orders where customerNumber = 144 or customerNumber = 128) as cust"

df=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                     user="root",\
                                     password="password",\
                                     url="jdbc:mysql://localhost:3306/classicmodels",\
                                     dbtable=query \
                                    ).load()
df.show()


+-----------+----------+------------+-----------+-------+--------------------+--------------+
|orderNumber| orderDate|requiredDate|shippedDate| status|            comments|customerNumber|
+-----------+----------+------------+-----------+-------+--------------------+--------------+
|      10101|2003-01-09|  2003-01-18| 2003-01-11|Shipped|Check on availabi...|           128|
|      10230|2004-03-15|  2004-03-24| 2004-03-20|Shipped|Customer very con...|           128|
|      10300|2003-10-04|  2003-10-13| 2003-10-09|Shipped|                null|           128|
|      10323|2004-11-05|  2004-11-12| 2004-11-09|Shipped|                null|           128|
|      10112|2003-03-24|  2003-04-03| 2003-03-29|Shipped|Customer requeste...|           144|
|      10320|2004-11-03|  2004-11-13| 2004-11-07|Shipped|                null|           144|
|      10326|2004-11-09|  2004-11-16| 2004-11-10|Shipped|                null|           144|
|      10334|2004-11-19|  2004-11-28|       null|On Hold|The